In [1]:
import numpy as np
import nltk
from os import getcwd
import numpy as np
import pandas as pd

# Preprocessing

In [2]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_german = stopwords.words('german')
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_german and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [3]:
stopwords_german = stopwords.words('german')

In [4]:
stopwords_german

['aber',
 'alle',
 'allem',
 'allen',
 'aller',
 'alles',
 'als',
 'also',
 'am',
 'an',
 'ander',
 'andere',
 'anderem',
 'anderen',
 'anderer',
 'anderes',
 'anderm',
 'andern',
 'anderr',
 'anders',
 'auch',
 'auf',
 'aus',
 'bei',
 'bin',
 'bis',
 'bist',
 'da',
 'damit',
 'dann',
 'der',
 'den',
 'des',
 'dem',
 'die',
 'das',
 'dass',
 'daß',
 'derselbe',
 'derselben',
 'denselben',
 'desselben',
 'demselben',
 'dieselbe',
 'dieselben',
 'dasselbe',
 'dazu',
 'dein',
 'deine',
 'deinem',
 'deinen',
 'deiner',
 'deines',
 'denn',
 'derer',
 'dessen',
 'dich',
 'dir',
 'du',
 'dies',
 'diese',
 'diesem',
 'diesen',
 'dieser',
 'dieses',
 'doch',
 'dort',
 'durch',
 'ein',
 'eine',
 'einem',
 'einen',
 'einer',
 'eines',
 'einig',
 'einige',
 'einigem',
 'einigen',
 'einiger',
 'einiges',
 'einmal',
 'er',
 'ihn',
 'ihm',
 'es',
 'etwas',
 'euer',
 'eure',
 'eurem',
 'euren',
 'eurer',
 'eures',
 'für',
 'gegen',
 'gewesen',
 'hab',
 'habe',
 'haben',
 'hat',
 'hatte',
 'hatten',
 '

In [5]:
df = pd.read_csv("parsed_protocols/27_016.csv", sep="@@", header=None)
df.head()

C:\Users\Stefan\Miniconda3\envs\protocol_parser\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,0,1
0,Präsident Mag. Wolfgang Sobotka,Meine sehr geehrten Damen und Herren Abge­ordn...
1,Präsident Mag. Wolfgang Sobotka,"Ich begrüße die Vertreter der Bundesregierung,..."
2,Präsident Mag. Wolfgang Sobotka,Als <b><i>verhindert</i></b> gemeldet sind für...
3,Präsident Mag. Wolfgang Sobotka,"Ich darf bekannt geben, dass die Sitzung von O..."
4,Präsident Mag. Wolfgang Sobotka,Um die Punkte 1 bis 4 der Tagesordnung in Verh...


In [6]:
df.iloc[:,1].to_numpy()

array(['Meine sehr geehrten Damen und Herren Abge\xadordnete, ich <b><i>eröffne</i></b> die 16.\xa0Sitzung des Nationalrates. Ich darf Sie an diesem außer\xadgewöhnlichen Tag, dem 15.\xa0März, herzlich begrüßen und Sie nochmals darauf hin\xadwei\xadsen, alle hygienischen Maßnahmen und Vorkehrungen einzuhalten und auch die unter anderem an den Eingängen aufgestellten Desinfektionsmittelspender zu nutzen. Ich bitte auch die Mitarbeiterinnen und Mitarbeiter und vor allem auch die Medienleute das in der gleichen Art und Weise zu tun.',
       'Ich begrüße die Vertreter der Bundesregierung, an der Spitze Bundeskanzler und Vize\xadkanzler, und ich begrüße sehr herzlich die Zuseherinnen und Zuseher zu Hause. Ich darf nochmals erklären, dass wir wie bereits in der gestrigen Sitzung in einem erwei\xad<span style="letter-spacing:.02em">terten Abstand sitzen und dass wir uns am Ende der heutigen Sitzung, wenn die</span> Abstimmungen stattfinden, kurz auf den Plätzen einfinden. Die Kolleginnen und

In [7]:
all_positive_tweets = df.iloc[:200,1].to_numpy()
all_negative_tweets = df.iloc[200:,1].to_numpy()

# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[int(all_positive_tweets.size*0.8):]
train_pos = all_positive_tweets[:int(all_positive_tweets.size*0.8)]
test_neg = all_negative_tweets[int(all_negative_tweets.size*0.8):]
train_neg = all_negative_tweets[:int(all_negative_tweets.size*0.8)]

In [8]:
train_x = np.concatenate((train_pos, train_neg)) 
test_x = np.concatenate((test_pos, test_neg))

# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# Define Logistic Regression

In [9]:
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''

    h = 1 / (1 + np.exp(-z))   
    return h

In [10]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''

    m = np.shape(x)[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = 1 / (1 + np.exp(-z))
        
        # calculate the cost function
        J = -1/m * (np.dot(np.transpose(y),np.log(h)) + np.dot(np.transpose(1-y),np.log(1-h)))

        # update the weights theta
        theta = theta - alpha/m * (np.dot(np.transpose(x),(h-y)))
        
    J = float(J)
    return J, theta

# Define Feature Extraction

In [11]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
        
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word,1), 0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0), 0)

    assert(x.shape == (1, 3))
    return x

# Train the Model

In [12]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.68611081.
The resulting vector of weights is [-0.0, 7.351e-05, -7.132e-05]


# Test the Model

In [13]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''

    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred

In [14]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy = np.sum(np.reshape(y_hat,(-1,1))==test_y)/test_y.shape[0]
    
    return accuracy

In [15]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.6625


# Arbitrary Predictions

In [16]:
my_tweet = 'This is a ridiculously good movie. The plot was wonderful and I was hooked until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['thi', 'is', 'a', 'ridicul', 'good', 'movi', 'the', 'plot', 'wonder', 'and', 'i', 'hook', 'until', 'the', 'end']
[[0.49999716]]
Negative sentiment
